# Feature Store Demo for Self-Healing Platform

## Overview
This notebook demonstrates how to implement a feature store for the OpenShift AIOps Self-Healing Platform using Parquet files for efficient storage and data versioning. It covers feature engineering, storage, retrieval, and versioning patterns essential for ML operations.

## Prerequisites
- Persistent storage configured (ODF)
- Processed data from metrics and events collection
- PyArrow and Pandas for Parquet operations
- Access to coordination engine for feature serving

## Expected Outcomes
- Understand feature store architecture and benefits
- Implement efficient Parquet-based feature storage
- Demonstrate data versioning and lineage tracking
- Create reusable feature engineering pipelines

## References
- ADR-012: Notebook Architecture for End-to-End Workflows
- ADR-013: Data Collection and Preprocessing Workflows
- Feature Store Best Practices Documentation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
import sys
import os
from pathlib import Path
import hashlib
from typing import Dict, List, Optional, Any, Tuple

# Parquet and data processing
try:
    import pyarrow as pa
    import pyarrow.parquet as pq
    parquet_available = True
    print("✅ PyArrow available for Parquet operations")
except ImportError:
    parquet_available = False
    print("⚠️ PyArrow not available - using pandas parquet engine")

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]

    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)

    # Fallback: search upward from cwd
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent

    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import (
        setup_environment, print_environment_info,
        save_processed_data, load_processed_data,
        validate_data_quality
    )
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    print("   Using minimal fallback implementations")

    # Minimal fallback implementations
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/data/feature_store', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {
            'data_dir': '/opt/app-root/src/data',
            'models_dir': '/opt/app-root/src/models',
            'working_dir': os.getcwd()
        }

    def print_environment_info(env_info):
        print(f"📁 Data dir: {env_info.get('data_dir', 'N/A')}")
        print(f"📁 Models dir: {env_info.get('models_dir', 'N/A')}")

    def save_processed_data(data, filename):
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        filepath = f'/opt/app-root/src/data/processed/{filename}'
        if filename.endswith('.parquet') and hasattr(data, 'to_parquet'):
            data.to_parquet(filepath)
        elif filename.endswith('.json'):
            with open(filepath, 'w') as f:
                if hasattr(data, 'items'):
                    serializable = {}
                    for k, v in data.items():
                        if hasattr(v, 'to_dict'):
                            serializable[k] = v.to_dict()
                        else:
                            serializable[k] = v
                    json.dump(serializable, f, default=str)
                else:
                    json.dump(data, f, default=str)
        print(f"💾 Saved: {filepath}")

    def load_processed_data(filename):
        filepath = f'/opt/app-root/src/data/processed/{filename}'
        if filename.endswith('.parquet'):
            return pd.read_parquet(filepath)
        elif filename.endswith('.json'):
            with open(filepath, 'r') as f:
                return json.load(f)
        return None

    def validate_data_quality(df, metric_name='unknown'):
        return {'valid': True, 'issues': [], 'metric_name': metric_name}

print("✅ Libraries imported successfully")

## 2. Setup & Configuration

Initialize the feature store and configure parameters for feature engineering and storage.

In [ ]:
# Set up environment
env_info = setup_environment()
print_environment_info(env_info)

# Feature Store Configuration
FEATURE_STORE_CONFIG = {
    'base_path': '/opt/app-root/src/data/feature_store',
    'versioning_enabled': True,
    'compression': 'snappy',  # Parquet compression
    'partition_cols': ['date', 'namespace'],  # Partitioning strategy
    'feature_groups': {
        'infrastructure_metrics': {
            'description': 'CPU, memory, disk, network metrics',
            'update_frequency': '5min',
            'retention_days': 90
        },
        'application_metrics': {
            'description': 'Request rates, response times, error rates',
            'update_frequency': '1min',
            'retention_days': 30
        },
        'cluster_events': {
            'description': 'Kubernetes events and state changes',
            'update_frequency': 'realtime',
            'retention_days': 60
        },
        'anomaly_features': {
            'description': 'Engineered features for anomaly detection',
            'update_frequency': '15min',
            'retention_days': 180
        }
    }
}

# Create feature store directory structure
feature_store_path = Path(FEATURE_STORE_CONFIG['base_path'])
feature_store_path.mkdir(parents=True, exist_ok=True)

for group_name in FEATURE_STORE_CONFIG['feature_groups'].keys():
    group_path = feature_store_path / group_name
    group_path.mkdir(exist_ok=True)
    
    # Create metadata directory
    (group_path / 'metadata').mkdir(exist_ok=True)
    
    # Create versions directory
    (group_path / 'versions').mkdir(exist_ok=True)

print(f"📁 Feature store initialized at: {feature_store_path}")
print(f"🗂️ Feature groups: {', '.join(FEATURE_STORE_CONFIG['feature_groups'].keys())}")

## 3. Define Feature Store Class

Define the FeatureStore class that manages feature groups, versioning, and metadata for ML models.

In [ ]:
class FeatureStore:
    """
    Simple feature store implementation using Parquet files
    """
    
    def __init__(self, base_path: str, config: Dict[str, Any]):
        self.base_path = Path(base_path)
        self.config = config
        self.feature_groups = config['feature_groups']
        
    def create_feature_group(self, name: str, schema: Dict[str, str], description: str = ""):
        """
        Create a new feature group with schema definition
        """
        group_path = self.base_path / name
        group_path.mkdir(exist_ok=True)
        
        # Save schema metadata
        metadata = {
            'name': name,
            'description': description,
            'schema': schema,
            'created_at': datetime.now().isoformat(),
            'version': '1.0.0'
        }
        
        metadata_path = group_path / 'metadata' / 'schema.json'
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ Created feature group: {name}")
        return group_path
    
    def write_features(self, group_name: str, features_df: pd.DataFrame, 
                      version: Optional[str] = None) -> str:
        """
        Write features to the feature store with versioning
        """
        if group_name not in self.feature_groups:
            raise ValueError(f"Feature group {group_name} not found")
        
        group_path = self.base_path / group_name
        
        # Generate version if not provided
        if version is None:
            version = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Add metadata columns
        features_df = features_df.copy()
        features_df['_version'] = version
        features_df['_ingestion_time'] = datetime.now()
        
        # Add partitioning columns if missing
        if 'date' not in features_df.columns:
            features_df['date'] = features_df.get('timestamp', datetime.now()).dt.date
        if 'namespace' not in features_df.columns:
            features_df['namespace'] = features_df.get('namespace', 'default')
        
        # Write to versioned location
        version_path = group_path / 'versions' / f'v_{version}.parquet'
        
        if parquet_available:
            # Use PyArrow for better performance
            table = pa.Table.from_pandas(features_df)
            pq.write_table(table, version_path, compression=self.config['compression'])
        else:
            # Fallback to pandas
            features_df.to_parquet(version_path, compression=self.config['compression'])
        
        # Update latest symlink
        latest_path = group_path / 'latest.parquet'
        if latest_path.exists():
            latest_path.unlink()
        
        # Create symlink to latest version (fallback to copy if symlink fails)
        try:
            latest_path.symlink_to(version_path)
        except OSError:
            # Fallback: copy file if symlink not supported
            features_df.to_parquet(latest_path, compression=self.config['compression'])
        
        # Update metadata
        self._update_metadata(group_name, version, len(features_df))
        
        print(f"✅ Written {len(features_df)} features to {group_name} (version: {version})")
        return version
    
    def read_features(self, group_name: str, version: Optional[str] = None, 
                     filters: Optional[List[Tuple]] = None) -> pd.DataFrame:
        """
        Read features from the feature store
        """
        if group_name not in self.feature_groups:
            raise ValueError(f"Feature group {group_name} not found")
        
        group_path = self.base_path / group_name
        
        if version is None:
            # Read latest version
            file_path = group_path / 'latest.parquet'
        else:
            # Read specific version
            file_path = group_path / 'versions' / f'v_{version}.parquet'
        
        if not file_path.exists():
            raise FileNotFoundError(f"Features not found: {file_path}")
        
        # Read with optional filters
        if parquet_available and filters:
            df = pq.read_table(file_path, filters=filters).to_pandas()
        else:
            df = pd.read_parquet(file_path)
            
            # Apply filters manually if PyArrow not available
            if filters:
                for filter_condition in filters:
                    column, operator, value = filter_condition
                    if operator == '==':
                        df = df[df[column] == value]
                    elif operator == '>=':
                        df = df[df[column] >= value]
                    elif operator == '<=':
                        df = df[df[column] <= value]
        
        print(f"✅ Read {len(df)} features from {group_name}")
        return df
    
    def _update_metadata(self, group_name: str, version: str, record_count: int):
        """
        Update feature group metadata
        """
        group_path = self.base_path / group_name
        metadata_path = group_path / 'metadata' / 'versions.json'
        
        # Load existing metadata
        if metadata_path.exists():
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
        else:
            metadata = {'versions': []}
        
        # Add new version info
        version_info = {
            'version': version,
            'created_at': datetime.now().isoformat(),
            'record_count': record_count,
            'file_path': f'versions/v_{version}.parquet'
        }
        
        metadata['versions'].append(version_info)
        metadata['latest_version'] = version
        metadata['total_versions'] = len(metadata['versions'])
        
        # Save updated metadata
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
    
    def list_versions(self, group_name: str) -> List[Dict[str, Any]]:
        """
        List all versions of a feature group
        """
        group_path = self.base_path / group_name
        metadata_path = group_path / 'metadata' / 'versions.json'
        
        if not metadata_path.exists():
            return []
        
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        return metadata.get('versions', [])

# Initialize feature store
feature_store = FeatureStore(
    FEATURE_STORE_CONFIG['base_path'], 
    FEATURE_STORE_CONFIG
)

print("🏪 Feature store initialized successfully")

## 4. Define Feature Engineering Functions

Define functions to engineer features from raw metrics, events, and logs for machine learning models.

In [ ]:
def generate_sample_features():
    """
    Generate sample feature data for demonstration
    """
    print("🎭 Generating sample feature data...")
    
    # Generate infrastructure metrics features
    timestamps = pd.date_range(
        start=datetime.now() - timedelta(hours=24),
        end=datetime.now(),
        freq='5min'
    )
    
    infrastructure_features = []
    namespaces = ['self-healing-platform', 'openshift-monitoring', 'default']
    nodes = ['node-1', 'node-2', 'node-3']
    
    for timestamp in timestamps:
        for namespace in namespaces:
            for node in nodes:
                # Generate realistic metrics with some patterns
                base_cpu = 30 + 20 * np.sin(timestamp.hour * np.pi / 12)  # Daily pattern
                base_memory = 60 + 15 * np.sin(timestamp.hour * np.pi / 8)  # Different pattern
                
                infrastructure_features.append({
                    'timestamp': timestamp,
                    'namespace': namespace,
                    'node': node,
                    'cpu_usage_percent': max(0, min(100, base_cpu + np.random.normal(0, 5))),
                    'memory_usage_percent': max(0, min(100, base_memory + np.random.normal(0, 8))),
                    'disk_usage_percent': np.random.uniform(20, 80),
                    'network_bytes_in': np.random.exponential(1000000),
                    'network_bytes_out': np.random.exponential(800000),
                    'pod_count': np.random.randint(5, 25),
                    'container_restarts': np.random.poisson(0.1)
                })
    
    infra_df = pd.DataFrame(infrastructure_features)
    
    # Generate application metrics features
    app_timestamps = pd.date_range(
        start=datetime.now() - timedelta(hours=6),
        end=datetime.now(),
        freq='1min'
    )
    
    application_features = []
    services = ['coordination-engine', 'mcp-server', 'prometheus']
    
    for timestamp in app_timestamps:
        for service in services:
            # Generate realistic application metrics
            base_requests = 100 + 50 * np.sin(timestamp.hour * np.pi / 12)
            
            application_features.append({
                'timestamp': timestamp,
                'namespace': 'self-healing-platform',
                'service': service,
                'requests_per_minute': max(0, base_requests + np.random.normal(0, 20)),
                'response_time_p95': np.random.lognormal(4, 0.5),  # Log-normal distribution
                'error_rate_percent': max(0, np.random.exponential(0.5)),
                'active_connections': np.random.randint(10, 100),
                'queue_depth': np.random.poisson(2),
                'cache_hit_rate': np.random.uniform(0.7, 0.95)
            })
    
    app_df = pd.DataFrame(application_features)
    
    print(f"✅ Generated {len(infra_df)} infrastructure feature records")
    print(f"✅ Generated {len(app_df)} application feature records")
    
    return infra_df, app_df

def engineer_anomaly_features(infra_df, app_df):
    """
    Engineer features specifically for anomaly detection
    """
    print("🔧 Engineering anomaly detection features...")
    
    anomaly_features = []
    
    # Group by time windows for feature engineering
    infra_hourly = infra_df.groupby([pd.Grouper(key='timestamp', freq='1H'), 'namespace']).agg({
        'cpu_usage_percent': ['mean', 'std', 'max'],
        'memory_usage_percent': ['mean', 'std', 'max'],
        'network_bytes_in': ['sum', 'mean'],
        'container_restarts': 'sum',
        'pod_count': 'mean'
    }).reset_index()
    
    # Flatten column names
    infra_hourly.columns = ['timestamp', 'namespace'] + [
        f"{col[0]}_{col[1]}" for col in infra_hourly.columns[2:]
    ]
    
    app_hourly = app_df.groupby([pd.Grouper(key='timestamp', freq='1H'), 'namespace']).agg({
        'requests_per_minute': ['sum', 'mean'],
        'response_time_p95': ['mean', 'max'],
        'error_rate_percent': ['mean', 'max'],
        'active_connections': 'mean'
    }).reset_index()
    
    # Flatten column names
    app_hourly.columns = ['timestamp', 'namespace'] + [
        f"{col[0]}_{col[1]}" for col in app_hourly.columns[2:]
    ]
    
    # Merge infrastructure and application features
    merged_df = pd.merge(infra_hourly, app_hourly, on=['timestamp', 'namespace'], how='outer')
    
    # Engineer additional features
    merged_df['cpu_memory_ratio'] = merged_df['cpu_usage_percent_mean'] / (merged_df['memory_usage_percent_mean'] + 1)
    merged_df['network_intensity'] = merged_df['network_bytes_in_sum'] / (merged_df['pod_count_mean'] + 1)
    merged_df['error_to_request_ratio'] = merged_df['error_rate_percent_mean'] / (merged_df['requests_per_minute_mean'] + 1)
    merged_df['resource_pressure'] = (merged_df['cpu_usage_percent_mean'] + merged_df['memory_usage_percent_mean']) / 2
    
    # Add time-based features
    merged_df['hour_of_day'] = merged_df['timestamp'].dt.hour
    merged_df['day_of_week'] = merged_df['timestamp'].dt.dayofweek
    merged_df['is_weekend'] = merged_df['day_of_week'].isin([5, 6]).astype(int)
    merged_df['is_business_hours'] = merged_df['hour_of_day'].between(9, 17).astype(int)
    
    print(f"✅ Engineered {len(merged_df)} anomaly feature records with {len(merged_df.columns)} features")
    return merged_df

# Generate sample data
infra_df, app_df = generate_sample_features()
anomaly_df = engineer_anomaly_features(infra_df, app_df)

print(f"\n📊 Sample Data Summary:")
print(f"Infrastructure features: {infra_df.shape}")
print(f"Application features: {app_df.shape}")
print(f"Anomaly features: {anomaly_df.shape}")

## 5. Create Feature Groups and Store Data

Create feature groups from sample data and demonstrate how to store and version features in the feature store.

In [ ]:
# Create feature groups and write data
print("🏪 Writing features to feature store...")

# Create feature group schemas
infra_schema = {
    'timestamp': 'datetime64[ns]',
    'namespace': 'string',
    'node': 'string',
    'cpu_usage_percent': 'float64',
    'memory_usage_percent': 'float64',
    'disk_usage_percent': 'float64',
    'network_bytes_in': 'float64',
    'network_bytes_out': 'float64',
    'pod_count': 'int64',
    'container_restarts': 'int64'
}

app_schema = {
    'timestamp': 'datetime64[ns]',
    'namespace': 'string',
    'service': 'string',
    'requests_per_minute': 'float64',
    'response_time_p95': 'float64',
    'error_rate_percent': 'float64',
    'active_connections': 'int64',
    'queue_depth': 'int64',
    'cache_hit_rate': 'float64'
}

# Create feature groups
feature_store.create_feature_group(
    'infrastructure_metrics', 
    infra_schema, 
    'CPU, memory, disk, network metrics from cluster nodes'
)

feature_store.create_feature_group(
    'application_metrics', 
    app_schema, 
    'Request rates, response times, error rates from applications'
)

# Write features with versioning
infra_version = feature_store.write_features('infrastructure_metrics', infra_df)
app_version = feature_store.write_features('application_metrics', app_df)
anomaly_version = feature_store.write_features('anomaly_features', anomaly_df)

print(f"\n📝 Feature versions created:")
print(f"- Infrastructure: {infra_version}")
print(f"- Application: {app_version}")
print(f"- Anomaly: {anomaly_version}")

## 6. Demonstrate Feature Retrieval and Filtering

Show how to retrieve features from the store, filter by time range, and access feature metadata.

In [ ]:
# Demonstrate feature retrieval and filtering
print("🔍 Demonstrating feature retrieval...")

# Read latest features
latest_infra = feature_store.read_features('infrastructure_metrics')
print(f"Latest infrastructure features: {len(latest_infra)} records")

# Read with filters (if PyArrow available)
if parquet_available:
    # Filter by namespace
    filtered_features = feature_store.read_features(
        'infrastructure_metrics',
        filters=[('namespace', '==', 'self-healing-platform')]
    )
    print(f"Filtered features (self-healing-platform): {len(filtered_features)} records")
else:
    # Manual filtering
    filtered_features = latest_infra[latest_infra['namespace'] == 'self-healing-platform']
    print(f"Filtered features (self-healing-platform): {len(filtered_features)} records")

# List versions
infra_versions = feature_store.list_versions('infrastructure_metrics')
print(f"\n📚 Infrastructure metrics versions: {len(infra_versions)}")
for version in infra_versions[-3:]:  # Show last 3 versions
    print(f"  - {version['version']}: {version['record_count']} records ({version['created_at']})")

# Demonstrate time-based analysis
print("\n📈 Feature analysis:")
print(f"CPU usage stats: mean={latest_infra['cpu_usage_percent'].mean():.1f}%, std={latest_infra['cpu_usage_percent'].std():.1f}%")
print(f"Memory usage stats: mean={latest_infra['memory_usage_percent'].mean():.1f}%, std={latest_infra['memory_usage_percent'].std():.1f}%")
print(f"Container restarts: total={latest_infra['container_restarts'].sum()}, max={latest_infra['container_restarts'].max()}")

## 7. Visualize Feature Store Data

Create visualizations of stored features to understand data distributions and feature engineering results.

In [ ]:
# Visualize feature store data
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Feature Store Data Analysis', fontsize=16, fontweight='bold')

# CPU usage over time by namespace
for namespace in latest_infra['namespace'].unique():
    ns_data = latest_infra[latest_infra['namespace'] == namespace]
    ns_hourly = ns_data.groupby(ns_data['timestamp'].dt.hour)['cpu_usage_percent'].mean()
    axes[0,0].plot(ns_hourly.index, ns_hourly.values, marker='o', label=namespace)

axes[0,0].set_title('CPU Usage by Hour and Namespace')
axes[0,0].set_xlabel('Hour of Day')
axes[0,0].set_ylabel('CPU Usage %')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Memory vs CPU scatter plot
scatter = axes[0,1].scatter(
    latest_infra['cpu_usage_percent'], 
    latest_infra['memory_usage_percent'],
    c=latest_infra['container_restarts'], 
    cmap='Reds', 
    alpha=0.6
)
axes[0,1].set_title('CPU vs Memory Usage (colored by restarts)')
axes[0,1].set_xlabel('CPU Usage %')
axes[0,1].set_ylabel('Memory Usage %')
plt.colorbar(scatter, ax=axes[0,1], label='Container Restarts')

# Application metrics if available
if len(app_df) > 0:
    app_df.groupby('service')['requests_per_minute'].mean().plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Average Requests per Minute by Service')
    axes[1,0].set_xlabel('Service')
    axes[1,0].set_ylabel('Requests/min')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Error rate distribution
    axes[1,1].hist(app_df['error_rate_percent'], bins=20, alpha=0.7, color='orange')
    axes[1,1].set_title('Error Rate Distribution')
    axes[1,1].set_xlabel('Error Rate %')
    axes[1,1].set_ylabel('Frequency')
else:
    axes[1,0].text(0.5, 0.5, 'No Application\nMetrics Available', 
                   ha='center', va='center', transform=axes[1,0].transAxes)
    axes[1,1].text(0.5, 0.5, 'No Application\nMetrics Available', 
                   ha='center', va='center', transform=axes[1,1].transAxes)

plt.tight_layout()
plt.show()

# Save feature store metadata
feature_store_summary = {
    'feature_groups': list(FEATURE_STORE_CONFIG['feature_groups'].keys()),
    'total_features': {
        'infrastructure_metrics': len(latest_infra),
        'application_metrics': len(app_df),
        'anomaly_features': len(anomaly_df)
    },
    'versions_created': {
        'infrastructure_metrics': infra_version,
        'application_metrics': app_version,
        'anomaly_features': anomaly_version
    },
    'storage_format': 'parquet',
    'compression': FEATURE_STORE_CONFIG['compression'],
    'created_at': datetime.now().isoformat()
}

save_processed_data(feature_store_summary, 'feature_store_summary.json')

print("\n💾 Feature store demo completed successfully!")
print("\n🎯 Next Steps:")
print("1. Integrate feature store with ML training pipelines")
print("2. Set up automated feature engineering jobs")
print("3. Implement feature monitoring and drift detection")
print("4. Create feature serving endpoints for real-time inference")
print("\n📚 Related Notebooks:")
print("- 02-anomaly-detection/: Use features for ML model training")
print("- 04-model-serving/: Serve features for real-time inference")